In [2]:
import numpy as np
import pandas as pd
import sklearn
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision.models import vgg19
vgg_net = vgg19(pretrained=True)
import tqdm
from tqdm import tqdm

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from facenet_pytorch import MTCNN, InceptionResnetV1

num_classes = 31

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
resnet = InceptionResnetV1(pretrained='vggface2', classify = True, num_classes=num_classes)
resnet

InceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_4a): 

In [4]:
resnet.load_state_dict(torch.load('resnet_model_data_augmentation.pth'))

# Set the model to evaluation mode
resnet.eval()


KeyboardInterrupt: 

In [ ]:
from facenet_pytorch import MTCNN
import torch
import numpy as np
import mmcv, cv2
from PIL import Image, ImageDraw
from IPython import display

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [ ]:
mtcnn = MTCNN(keep_all=True, device=device)

In [ ]:
video = mmcv.VideoReader('/home/bishal/demoprojects/facerecognition/Datasets/Lisa_and_Ellen.mp4')
frames = [Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) for i, frame in enumerate(video) if i < 500]

display.Video('video.mp4', width=640, embed=True)

In [ ]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, SubsetRandomSampler
from torchvision.datasets import ImageFolder
from sklearn.model_selection import train_test_split

data_dir = '/home/bishal/demoprojects/facerecognition/Datasets/Faces_small'

transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

])

dataset = ImageFolder(root=data_dir, transform=transform)
dataset.class_to_idx
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}




In [ ]:
from PIL import Image, ImageDraw, ImageFont
import torch 


detection_threshold = 0.9
recognition_threshold = 0.9

softmax = torch.nn.Softmax(dim=1).eval()
resnet.eval()
print(len(frames))
frames_tracked = []
for i, frame in enumerate(frames):
    print('\rTracking frame: {}'.format(i + 1), end='')
    
    # Detect faces
    # boxes, _ = mtcnn.detect(frame)

    # Detect faces
    boxes, batch_probs, batch_points = mtcnn.detect(frame, landmarks=True)

    # boxes = boxes.tolist()
    
    # filtering detected based on their probability

    # if boxes is not None:
    #     for i, det_prob in enumerate(batch_probs):
    #         if det_prob < detection_threshold:
    #             boxes.pop(i)
    #     if len(boxes) == 0:
    #         boxes = None
    #     else:
    #         boxes = np.array(boxes)

    # Extract faces
    faces = mtcnn.extract(frame, boxes, save_path=None) # of shape Bx3x160x160

    
    # Select faces
    # boxes, batch_probs, batch_points = mtcnn.select_boxes(
    #     boxes, batch_probs, batch_points, frame, method="probability", threshold=0.9)
    
    # print(batch_boxes.shape)

     


    # Draw faces and labels
    frame_draw = frame.copy()
    draw = ImageDraw.Draw(frame_draw)


    font = ImageFont.load_default(size=30.0)

    if boxes is not None:

        # resize Bx3x160x160 to Bx3x224x224

        # Define the transformation to convert a tensor to a PIL image
        # to_pil_image = transforms.ToPILImage()

        # # Loop over each image in the batch and convert to PIL image
        # pil_images = [to_pil_image(faces[i]) for i in range(faces.size(0))]

        # new_faces_list = []
        # for pil_image in pil_images:
        #     new_faces_list.append(transform(pil_image))

        # faces = torch.stack(new_faces_list)

        # perform recognition over detected faces
        outputs = resnet(faces)
        prob = softmax(outputs)

        predicted_prob, predicted = torch.max(prob.detach(), 1)
        predicted_prob = predicted_prob.cpu().numpy()
        predicted = predicted.cpu().numpy()

        print(predicted_prob)

        # Assuming `predicted` is a list of class indices and `boxes` is a list of bounding boxes
        labels = [dataset.idx_to_class[pred] for pred in predicted]

        
        for i, (box, label) in enumerate(zip(boxes, labels)):
            if predicted_prob[i] > recognition_threshold and batch_probs[i] > detection_threshold:
                draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
                
                # Add label
                text_size = draw.textbbox((0, 0), str(int(predicted_prob[i]*100))+label, font=font)
                text_location = [box[0], box[1] - text_size[1]]
                textbox_location = [box[0], box[1] - text_size[1], box[0] + text_size[0], box[1]]
                
                draw.rectangle(textbox_location, fill=(255, 0, 0))
                draw.text(text_location, str(int(predicted_prob[i]*100))+label, fill=(255, 255, 255), font=font)

    # Add to frame list
    frames_tracked.append(frame_draw.resize((640, 360), Image.BILINEAR))
print('\nDone')

dim = frames_tracked[0].size
fourcc = cv2.VideoWriter_fourcc(*'FMP4')    
video_tracked = cv2.VideoWriter('video_tracked_1.mp4', fourcc, 25.0, dim)
for frame in frames_tracked:
    video_tracked.write(cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR))
video_tracked.release()


500
Tracking frame: 1[0.98786235]
Tracking frame: 2[0.9905076]
Tracking frame: 3[0.9641175]
Tracking frame: 4[0.98959863]
Tracking frame: 5[0.88424873]
Tracking frame: 6[0.8545502]
Tracking frame: 7[0.8781823]
Tracking frame: 8[0.8138625]
Tracking frame: 9[0.6123862]
Tracking frame: 10[0.94671255]
Tracking frame: 11[0.99527407]
Tracking frame: 12[0.9975618  0.36523405]
Tracking frame: 13[0.99301404]
Tracking frame: 14[0.9980573]
Tracking frame: 15[0.99783033]
Tracking frame: 16[0.9964097]
Tracking frame: 17[0.5127602 0.9971718]
Tracking frame: 18[0.98739445]
Tracking frame: 19[0.9897111]
Tracking frame: 20[0.98665667]
Tracking frame: 21[0.97426856]
Tracking frame: 22[0.85385865]
Tracking frame: 23[0.9258421]
Tracking frame: 24[0.9061773]
Tracking frame: 25[0.98245865]
Tracking frame: 26[0.80317885]
Tracking frame: 27[0.59504986]
Tracking frame: 28[0.43298903]
Tracking frame: 29[0.49965507]
Tracking frame: 30[0.5173736]
Tracking frame: 31[0.48126954]
Tracking frame: 32[0.6871057]
Tracki

OpenCV: FFMPEG: tag 0x34504d46/'FMP4' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
